# Plausability of Lottery Luck

#### [Dylan D. Daniels](http://statistics.berkeley.edu/people/dylan-david-daniels) and [Philip B. Stark](www.stat.berkeley.edu/~stark), Department of Statistics, University of California, Berkeley
#### Based on MATLAB code by [Skip Garibaldi](http://www.garibaldibros.com/)

This tool appraises whether it is plausible that a given individual won a set of lottery prizes honestly. 

The code reads a comma-separated values file (CSV) of wins and odds.

The user inputs an upper bound on the number of players (for instance, one might assume that the number of people playing the lottery isn't greater than the number of residents of the state), and a tiny "threshold" probability.

The code outputs a lower bound on the amount all those people would have had to spend for any of them to have a tiny chance of winning so often, where "tiny" is the threshold number chosen by the user.

If the required spending amount is, for example, several times the median house price in the state, it may call into question whether the winner won honestly.

The current version can analyze data for only one gambler at a time. 

The code implements the mathematics described in the first link below. The third link is to a public lecture about the method, and results for reported lottery winners in Florida. 
The fourth and fifth links are news stories that relied on such calculations.

See:
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck. _Mathematics Magazine_, _88_ 196–211. doi:10.4169/math.mag.88.3.196.c, Reprint: http://www.stat.berkeley.edu/~stark/Preprints/luck15.pdf http://www.jstor.org/stable/10.4169/math.mag.88.3.196
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck &hellip; or do they? _MAA Focus_, August/September, 37–38. http://www.maa.org/sites/default/files/pdf/MAAFocus/Focus_AugustSeptember_2015.pdf
+ https://www.youtube.com/watch?v=s8cHHWNblA4
+  Lottery odds: To win, you’d have to be a loser. Lawrence Mower, _Palm Beach Post_, 28 March 2014. http://www.mypalmbeachpost.com/news/news/lottery-odds-to-win-youd-have-to-be-a-loser/nfL57
+ Against all Odds, Gavin Off and Adam Bell, _The Charlotte Observer_, 29 September 2016.
http://www.charlotteobserver.com/news/special-reports/against-all-odds/


## Instructions:
1. Compile a CSV file for each gambler. The CSV file should contain three columns:  "Probability," "Number," and "Cost." 

Each row corresponds to one type of wager. "Probability" is the chance of winning that wager; "Number" is the number of times the gambler collected on that wager; and "Cost" is the cost per ticket or play on that wager. The computations assume that the gambler did not win any dependent bets, for instance, two bets on the same drawing.

2. Put the filename of your CSV file in the box below, along with the values of POPULATION and THRESHOLD.

3. On the toolbar of this browser window (under the jupyter logo), click "Cell" --> "Run All". Wait a bit for your results to appear at the bottom of this page. 

In [1]:
from __future__ import print_function, division

# Put the name of your CSV file here:
# CSV_FILENAME = 'FILL_ME_IN.csv'
# CSV_FILENAME = 'manning-edit.csv'
# CSV_FILENAME = 'pandya.csv'
CSV_FILENAME = 'havis.csv'

# set the population size and overall cutoff probability
POPULATION = 10**7   # population of North Carolina
THRESHOLD =  10**(-7) # one in ten million threshold

In [2]:
import numpy as np
from scipy.special import betainc
from scipy.optimize import minimize

def binTail(p, n, t): # upper tail probability for a vector of Binomial(n,p) random variables
    return betainc(n, t - n + 1, p)

def binTailln(p, n, t): # logarithm of the upper tail probability for a vector of Binomial(n,p) random variables
    return np.log(binTail(p, n, t))

def constraintFn(p, n): # constraint function: probability of vector of wins must be at least CUT
    return lambda x: np.sum(binTailln(p, n, x)) - np.log(CUT)

def objectiveFn(c):  # construct function that gives cost of vector x of bets, for cost-per-bet vector c
    return lambda x: np.dot(x, c)

def solve(x0, upperBoundVec, p, n, c, eps, debugMode, maxiter, method='SLSQP'):  
    # invoke the constrained optimizer
    # 
    #    x0:     starting guess
    #    p:      vector of game probabilities
    #    n:      vector of number of wins of each game
    #    c:      vector of game costs
    #    eps:    stepsize for Hessian approximation
    #    debugMode: True for verbose output
    #    maxiter: maximum iterations in optimizer
    #    method: underlying minimization algorithm
    #       
    cons = ({'type': 'ineq', 'fun': constraintFn(p, n)})   # overall probability constraint
    bnds = tuple((n[i], upperBoundVec[i]) for i in range(len(n)))  # must bet at least n times to win n times
    return minimize(objectiveFn(c), x0, method=method, jac=(lambda x: c),
                    constraints=cons, bounds=bnds,
                    options={'disp': debugMode, 'maxiter': maxiter, 'eps': eps})

def readCsv(filename):  # read the csv file of data for a player
    with open(CSV_FILENAME, 'r') as f:
        firstLine = f.readline()
        if firstLine.strip() != "Probability,Number,Cost":
            raise Exception('Data column headings should be "Probability, Number, Cost"')
    values = np.loadtxt(filename, dtype=np.float_, delimiter=',', skiprows=1)  # skip the header
    values = np.atleast_2d(values)
    pValues = values[:,0]
    nValues = values[:,1]
    cValues = values[:,2]
    return (pValues, nValues, cValues)

def solveProblem(tries=5, debugMode=False, epsilon = 1e-7, epsFac=8, maxiter=10**4):
    # Try up to epsFac values of the Hessian step size, related by powers of 10 (Hessian approximation step sizes)
    optimalValues = []     # candidate optima
    optimalProbs = []      # probabilities associated with those optima
    optimalSolutions = []  # detailed optimization output for candidate optima
    if debugMode:
        print("n: {} \np: {}".format(n,p))
    for meth in methods:   # try different optimization methods
        for epsIndex in range(epsFac):  # try different step sizes in the Hessian
            x0 = np.array(that/divisor) # starting guess
            for i in range(tries):
                while (np.sum(np.log(binTail(p, n, x0))) - np.log(CUT)) < 0:  # ensure x0 is a feasible point
                    x0 = np.add(x0,np.ones_like(x0))  # increment every element of x
                if (debugMode):
                    print("method: {} try: {} \nx0: {} \nprobability {}:".format(meth,i,x0,\
                                np.prod(binTail(p, n, x0))))
                optimOutput = solve(x0, that, p, n, c, epsilon*10**epsIndex, debugMode, maxiter, method=meth)
                if optimOutput['success']:
                    optimalValues.append(optimOutput['fun'])
                    attainedProb = np.prod(binTail(p, n, optimOutput['x']))
                    optimalProbs.append(attainedProb)
                    optimalSolutions.append(optimOutput)
                    if debugMode:
                        print(optimOutput)
                        print("attained probability: {}".format(attainedProb))
                x0 = [np.random.randint(low=n[i], high=that[i]) for i in range(len(n))] # update x0 randomly
    if len(optimalValues) == 0:
        raise Exception('No candidate optimal solution found.')
    bestValue = np.min(optimalValues)
    largestProb = np.max(tuple(optimalProbs))
    if debugMode:
        print("\nFound {} candidate minima: {}".format(len(optimalValues), optimalValues))
        print("Best value: {}".format(bestValue))
    print("Everyone in the population of {} people would have to spend at least ${:,} to have probability {} that at least one would win these bets so often."
          .format(POPULATION, np.int(bestValue), THRESHOLD))
    return optimalValues, optimalProbs

In [3]:
(p, n, c) = readCsv(CSV_FILENAME)  # read the data for the player
that = n/p  # expected number of wagers on each bet required to win that bet n times

debugMode = False  # verbose output if True; set to False for less output

np.random.RandomState(seed=1234567890) # setting seed explicitly, for reproducibility

if debugMode:
    print ("initial t_hat: {} \ninitialprobability: {}".format(that,np.prod(binTail(p, n, that))))

CUT = THRESHOLD / POPULATION # Bonferroni cutoff probability

# 'that' will be used as an upper bound; ensure that it's compatible with the probability constraint
while np.prod(binTail(p, n, that)) < CUT:
    that = 2*that
    
divisor = 5 # initial value for optimizer is expected number divided by divisor (modified to ensure feasibility)
methods = ['SLSQP','COBYLA']  # COBYLA will ignore the individual bounds, but should honor the probability constraint

if debugMode:
    print ("adjusted t_hat: {} \nadjusted probability: {}".format(that,np.prod(binTail(p, n, that))))

In [4]:
optimalValues, optimalProbs = solveProblem(tries = 5, debugMode=debugMode, epsilon = 1e-7, epsFac=8, maxiter=10**4)

//anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:381: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  RuntimeWarning)
//anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:400: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:33: OptimizeWarning: Unknown solver options: eps


Everyone in the population of 10000000 people would have to spend at least $1,903,524 to have probability 1e-07 that at least one would win these bets so often.


In [5]:
# version information
%load_ext version_information
%version_information scipy, numpy, pandas, matplotlib, notebook

//anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:809: FormatterWarning: JSON expects JSONable list/dict containers, not JSON strings
  FormatterWarning)


Software versions
Python 2.7.11 64bit [GCC 4.2.1 (Apple Inc. build 5577)]
IPython 5.1.0
OS Darwin 16.1.0 x86_64 i386 64bit
scipy 0.18.1
numpy 1.11.2
pandas 0.19.0
matplotlib 1.5.0
notebook 4.2.3
Tue Oct 25 18:38:00 2016 PDT